# 準備

python-colormath をインストールして下さい。たとえば `pip install colormath` でインストールできます。Anaconda にも含まれているかもしれません。

## ライブラリの読み込みとユーティリティメスッドの定義

In [1]:
from math import pi

import numpy as np

from colormath.color_objects import ColorBase, sRGBColor as sRGB, HSVColor as HSV, LabColor as Lab
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import *
from bokeh.layouts import *

output_notebook()

Loading BokehJS ...

In [2]:
# sRGB 関連のユーティリティ
sRGB.fromHex = lambda hex: sRGB.new_from_rgb_hex(hex)
sRGB.hex = lambda c: c.get_rgb_hex()
def _sRGB_clamp_(c):
    r, g, b = c.clamped_rgb_r, c.clamped_rgb_g, c.clamped_rgb_b
    return sRGB(r, g, b)
sRGB.clamp = _sRGB_clamp_
HSV.hex = Lab.hex = lambda c: c.sRGB().hex()
ColorBase.values = ColorBase.get_value_tuple

# 色変換のメソッド
ColorBase.sRGB = lambda c: convert_color(c, sRGB)
ColorBase.HSV  = lambda c: convert_color(c, HSV)
ColorBase.Lab  = lambda c: convert_color(c, Lab)

# 色距離を計算するメソッド
def delta_e_cie(c1, c2):
    if not isinstance(c1, Lab): c1 = c1.Lab()
    if not isinstance(c2, Lab): c2 = c2.Lab()
    return delta_e_cie2000(c1, c2)

ColorBase.delta = lambda c1, c2: delta_e_cie(c1, c2)

## 実験的に用いる sRGB 色の定義

In [3]:
colors = list(zip('白 赤 緑 青'.split(),
                  [sRGB.fromHex(hex)
                   for hex in '#ffffff #ff0000 #00ff00 #0000ff'.split()]))
print('# Standard RGB color space')
for name, color in colors: print(f'{name}\t{color}')

def plot(): return Plot(frame_width=400, frame_height=200,
                        x_range=Range1d(0, 400), y_range=Range1d(0, 200))

p = plot()
xs = np.linspace(50, 350, num=3, dtype=np.int)
for (_, color), x in zip(colors[1:], xs):
    p.add_glyph(Circle(x=x, y=100, radius=45, fill_color=color.hex(), line_color=None))
show(p)

# Standard RGB color space
白	sRGBColor (rgb_r:1.0000 rgb_g:1.0000 rgb_b:1.0000)
赤	sRGBColor (rgb_r:1.0000 rgb_g:0.0000 rgb_b:0.0000)
緑	sRGBColor (rgb_r:0.0000 rgb_g:1.0000 rgb_b:0.0000)
青	sRGBColor (rgb_r:0.0000 rgb_g:0.0000 rgb_b:1.0000)


## 色空間の変換

In [4]:
print('# HSV color space')
for name, color in colors: print(f'{name}\t{color.HSV()}')
    
p = plot()
for hue, x in zip(range(0, 360, 120), xs):
    p.add_glyph(Circle(x=x, y=100, radius=45, fill_color=HSV(hue, 1, 1).hex(), line_color=None))
show(p)

# HSV color space
白	HSVColor (hsv_h:0.0000 hsv_s:0.0000 hsv_v:1.0000)
赤	HSVColor (hsv_h:0.0000 hsv_s:1.0000 hsv_v:1.0000)
緑	HSVColor (hsv_h:120.0000 hsv_s:1.0000 hsv_v:1.0000)
青	HSVColor (hsv_h:240.0000 hsv_s:1.0000 hsv_v:1.0000)


In [5]:
print('# CIELab color space')
for name, color in colors: print(f'{name}\t{color.Lab()}')

labs60 = [Lab(60, 50 * np.cos(theta), 50 * np.sin(theta))
          for theta in np.linspace(0, 2 * pi, num=4)[:-1]]
    
p = plot()
for lab, x in zip(labs60, xs):
    p.add_glyph(Circle(x=x, y=100, radius=45, fill_color=lab.hex(), line_color=None))
    print(lab)
show(p)

# CIELab color space
白	LabColor (lab_l:100.0000 lab_a:-0.0005 lab_b:-0.0086)
赤	LabColor (lab_l:53.2390 lab_a:80.0905 lab_b:67.2014)
緑	LabColor (lab_l:87.7350 lab_a:-86.1829 lab_b:83.1795)
青	LabColor (lab_l:32.2994 lab_a:79.1914 lab_b:-107.8655)
LabColor (lab_l:60.0000 lab_a:50.0000 lab_b:0.0000)
LabColor (lab_l:60.0000 lab_a:-25.0000 lab_b:43.3013)
LabColor (lab_l:60.0000 lab_a:-25.0000 lab_b:-43.3013)


# 色差

In [6]:
(_, white), black = colors[0], sRGB(0, 0, 0)
print('# 明度')
for name, c in colors:
    print(f'{name}\t{c.Lab().lab_l:.2f}')

print('\n# 白との色差')
for name, c in colors:
    print(f'{name}\t{c.delta(white):.2f}')
print('\n# 黒との色差')
for name, c in colors:
    print(f'{name}\t{c.delta(black):.2f}')


# 明度
白	100.00
赤	53.24
緑	87.74
青	32.30

# 白との色差
白	0.00
赤	45.81
緑	33.26
青	64.03

# 黒との色差
白	100.00
赤	50.41
緑	87.86
青	39.68


In [7]:
print('# CIELab の場合')
gray0, gray100 = Lab(1, 0, 0), Lab(100, 0, 0)

for name, c in zip('赤 緑 青'.split(), labs60):
    print(f'{name}-白\t{c.delta(gray100):.2f}')
print()
for name, c in zip('赤 緑 青'.split(), labs60):
    print(f'{name}-黒\t{c.delta(gray0):.2f}')

print(f'\n赤-緑\t{labs60[0].delta(labs60[1]):.2f}')
print(f'緑-青\t{labs60[0].delta(labs60[1]):.2f}')
print(f'青-赤\t{labs60[0].delta(labs60[1]):.2f}')

# CIELab の場合
赤-白	37.32
緑-白	36.60
青-白	36.60

赤-黒	52.29
緑-黒	51.78
青-黒	51.78

赤-緑	59.35
緑-青	59.35
青-赤	59.35


In [8]:
labs = labs60.copy()

def CIELab_plot(doc):
    p = plot()
    circles = [Circle(x=x, y=100, radius=45, fill_color=lab.hex(), line_color=None)
               for lab, x in zip(labs, xs)]

    for circle in circles:
        p.add_glyph(circle)
    slider = Slider(start=0, end=100, step=1, value=60)
    def update(v):
        for i in range(len(circles)):
            l, a, b = labs[i].values()
            circles[i].fill_color = Lab(v, a, b).sRGB().clamp().hex()
    slider.on_change('value', lambda _attr, _old, v: update(v))
    text = Div(text='''L*値を高くしすぎると、sRGB空間では表現できない色が生成され、sRGB に変換したときに R/G/B が [0, 1] の範囲を外れます。
    このような困った場合のために、一旦、sRGBに変換後、R/G/B 値を無理矢理 [0, 1] の範囲に収めているのが <code>clamp()</code> です。
    この<code>clamp</code>処理は CIEL*a*b* で表現したのと異なる色が生成します。''')
    layout = column([p, slider, text])
    doc.add_root(layout)

show(CIELab_plot)

# 課題１

sRGB 的なグレースケールは `sRGB(g, g, g)` $g \in [0, 1]$ として与えられる。一方、CIEL*a*b* 的なグレースケールは `Lab(g, 0, 0)` $\in [0, 100]$ として与えられる。

- sRGB的なグレースケールとCIELab的なグレースケールを描画しなさい。

    グレースケールは16個（あるいは17個）の矩形タイルを横に並べたものに黒から白に変化する色を塗り潰すことで実現する。グレースケールの左端は黒、右端は白とし、そのほかのタイルの色は白、黒のタイルからの距離に応じて各色空間で自然な線型補間を施して得られる色で着色すること。

    タイルを塗り潰すには[Rect](https://docs.bokeh.org/en/latest/docs/reference/models/glyphs/rect.html) を使うとよい。

    タイルの縁取りは邪魔なので除去すること。

- 描画結果を目視、及び CIELab の色差のについて比較し、論じなさい。

In [9]:
# 課題１についてのプログラム

（課題１についての考察）

# 課題２

- sRGB 的に自然と思われる、黒→赤、黒→青に関する16階調二次元グラデーションを Bokeh を用いて描きなさい。
    
    $16x16$個のタイルを正方形に敷きつめたものについて、右下に赤、左下に黒、左上に青のタイルを配置し、赤-青の対角線から下のタイルたちに黒赤青の色を線型補間して彩色して得られる下三角行列様の結果が欲しい。
    
    タイルの縁取りは邪魔なので除去すること。

- CIEL*a*b* 的に自然と思われる、同様の方法で16階調二次元グラデーションを Bokeh を用いて描きなさい。ただし、黒、赤、青は sRGB で利用した色を CIELab に変換したものを利用すること。CIELab の L-, a-, b-値は、`lab.values()` によって取得できる。

- ふたつの出力を目視、及び CIELab 色差の計測にて比較論じなさい。

In [10]:
# 課題２についてのプログラム

（課題２についての考察）

# 応用課題（やらなくてよい）

課題２の結果を見て、CIELab の結果に少し失望したかもしれない。一部の領域で CIELab を用いたグラデーションが思ったほど良い結果でなかった理由を考察し、問題を明らかにしなさい。その上で、小さな工夫により満足のいく二次元グラデーションを作成しなさい。

# 提出方法

- 提出期限: 2/6

- 提出先 (Dropbox file request) https://www.dropbox.com/request/eQuoLjmDihLRLsRWgWTB